In [88]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


In [89]:
egx = pd.read_csv('egxsentiment.csv').drop(columns = ['Unnamed: 0','News']).rename(columns = {'Sentiment':'EGX_SENT'})
egx.count()

Date        221
EGX_SENT    221
dtype: int64

In [90]:
inv = pd.read_csv('investmentsentiment.csv').drop(columns = ['Unnamed: 0','News']).rename(columns = {'sentiment':'INV_SENT'})
inv.count()

Date        450
INV_SENT    299
dtype: int64

In [91]:
stock = pd.read_csv('EGX 30 Historical Data.csv').rename(columns = {'Change %':'Change'})

In [92]:
stock.drop("change", axis=1, inplace=True)
stock.head()

,Date,Price,Open,High,Low,Vol.,Change
0,03/26/2024,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%
1,03/25/2024,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%
2,03/24/2024,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%
3,03/21/2024,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%
4,03/20/2024,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%


In [93]:
stock['Date'] = pd.to_datetime(stock['Date'])
stock['Date'] = stock['Date'].dt.strftime('%Y-%m-%d')

egx['Date'] = pd.to_datetime(stock['Date'])
egx['Date'] = egx['Date'].dt.strftime('%Y-%m-%d')

inv['Date'] = pd.to_datetime(stock['Date']) 
inv['Date'] = inv['Date'].dt.strftime('%Y-%m-%d')
stock.head()

,Date,Price,Open,High,Low,Vol.,Change
0,2024-03-26,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%
1,2024-03-25,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%
2,2024-03-24,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%
3,2024-03-21,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%
4,2024-03-20,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%


In [94]:
df = stock.merge(egx, on='Date', how='left')
df = df.merge(inv, on='Date', how='left')
df.head()

,Date,Price,Open,High,Low,Vol.,Change,EGX_SENT,INV_SENT
0,2024-03-26,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98%,negative,positive
1,2024-03-25,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98%,positive,positive
2,2024-03-24,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02%,negative,positive
3,2024-03-21,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63%,negative,positive
4,2024-03-20,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30%,positive,neutral


In [95]:
df['EGX_SENT'] = df['EGX_SENT'].fillna('neutral')
df['INV_SENT']= df['INV_SENT'].fillna('neutral')

In [96]:
df['Change'] = df['Change'].str.replace('%', '').astype(float)

df

,Date,Price,Open,High,Low,Vol.,Change,EGX_SENT,INV_SENT
0,2024-03-26,"29,057.50","31,013.88","29,532.31","28,997.13",211.16M,-0.98,negative,positive
1,2024-03-25,"29,344.46","31,013.88","29,344.46","28,946.14",1.19M,0.98,positive,positive
2,2024-03-24,"29,060.02","31,013.88","29,377.43","28,777.22",1.75M,0.02,negative,positive
3,2024-03-21,"29,054.96","31,013.88","29,094.10","28,280.76",2.80M,1.63,negative,positive
4,2024-03-20,"28,589.37","28,675.70","29,229.67","28,542.99",177.70M,-0.30,positive,neutral
...,...,...,...,...,...,...,...,...,...
540,2022-01-09,"11,978.56","12,043.42","12,044.78","11,958.61",59.66M,-0.54,neutral,neutral
541,2022-01-05,"12,043.42","11,982.61","12,069.23","11,980.87",144.60M,0.51,neutral,neutral
542,2022-01-04,"11,982.61","11,897.40","11,984.21","11,897.15",114.48M,0.72,neutral,neutral
543,2022-01-03,"11,897.40","11,909.72","11,916.04","11,870.47",79.32M,-0.10,neutral,neutral


In [97]:
sentiment_map={'positive':1, 'neutral':0, 'negative': -1}
df['EGX_SENT'] = df['EGX_SENT'].map(sentiment_map)
df['INV_SENT'] = df['INV_SENT'].map(sentiment_map)

In [98]:
df['EGX_SENT'] = df['EGX_SENT'].shift(-1)
df['INV_SENT'] = df['INV_SENT'].shift(-1)

df = df[['Date','Price','Change','EGX_SENT','INV_SENT']]

In [99]:
import plotly.express as px

fig = px.line(stock, x="Date", y="Change", markers=True)
fig.show()

In [100]:
df= df.sort_values(by='Date')
df.reset_index(drop=True, inplace=True)

In [101]:
train = df[df['Date']<'2024-01-01']
test = df[df['Date']>='2024-01-01']

In [102]:
train.fillna(0,inplace=True)
train

C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\2246764937.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Price,Change,EGX_SENT,INV_SENT
0,2022-01-02,"11,909.72",-0.33,0.0,0.0
1,2022-01-03,"11,897.40",-0.10,0.0,0.0
2,2022-01-04,"11,982.61",0.72,0.0,0.0
3,2022-01-05,"12,043.42",0.51,0.0,0.0
4,2022-01-09,"11,978.56",-0.54,0.0,0.0
...,...,...,...,...,...
481,2023-12-25,"24,226.68",1.70,0.0,1.0
482,2023-12-26,"23,994.54",-0.96,0.0,0.0
483,2023-12-27,"24,351.12",1.49,1.0,0.0
484,2023-12-28,"24,691.43",1.40,1.0,0.0


In [103]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [104]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

In [105]:
#Here I'am performing hyper paramter tuning to get the best results
auto_model = auto_arima(train['Change'], 
                        seasonal=True, 
                        m=12,  # Seasonality period
                        stepwise=True, # this is to find the best parameters 
                        trace=True,
                        error_action='ignore', 
                        suppress_warnings=True)

# Extract the best parameters
best_params = auto_model.get_params()
print("Best Parameters:", best_params)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=4.37 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=1741.839, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=1739.198, Time=0.37 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=1740.061, Time=0.43 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=1745.862, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=1738.051, Time=0.12 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=1739.162, Time=0.32 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=1741.060, Time=1.29 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=1736.154, Time=0.19 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=1737.601, Time=0.44 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=1737.568, Time=0.39 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=1739.390, Time=1.00 sec
 ARIMA(3,0,0)(0,0,0)[12] intercept   : AIC=1736.842, Time=0.19 sec
 ARIMA(2,0,1)(0,0,0)[12] intercept   : AIC=1737.644, Time=0.44 sec
 ARIMA(1,0,1)(0,0,0)[12]

In [106]:
best_params = auto_model.get_params()
print("Best Parameters:", best_params)

Best Parameters: {'maxiter': 50, 'method': 'lbfgs', 'order': (2, 0, 0), 'out_of_sample_size': 0, 'scoring': 'mse', 'scoring_args': {}, 'seasonal_order': (0, 0, 0, 12), 'start_params': None, 'suppress_warnings': True, 'trend': None, 'with_intercept': True}


In [107]:
best_p = best_params['order'][0]
best_d = best_params['order'][1]
best_q = best_params['order'][2]
best_P = best_params['seasonal_order'][0]
best_D = best_params['seasonal_order'][1]
best_Q = best_params['seasonal_order'][2]
best_m = best_params['seasonal_order'][3]


In [108]:
exog_vars = ['INV_SENT', 'EGX_SENT']


In [109]:
model = SARIMAX(train['Change'],
                order=(best_p, best_d, best_q),
                seasonal_order=(best_P, best_D, best_Q, best_m),
                 exog=train[exog_vars]) 
results = model.fit(disp=False)
print(results.summary())


                               SARIMAX Results                                
Dep. Variable:                 Change   No. Observations:                  486
Model:               SARIMAX(2, 0, 0)   Log Likelihood                -864.566
Date:                Sun, 26 May 2024   AIC                           1739.131
Time:                        21:42:56   BIC                           1760.062
Sample:                             0   HQIC                          1747.354
                                - 486                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
INV_SENT       0.1679      0.164      1.027      0.305      -0.153       0.488
EGX_SENT       0.1805      0.143      1.262      0.207      -0.100       0.461
ar.L1          0.1028      0.043      2.416      0.0

In [110]:
sarimax_forecast = results.predict(start=test.index[0], end=test.index[-1], exog=test[exog_vars])


In [111]:
sarimax_forecast

486    3.365480e-02
487    1.133389e-01
488    1.859056e-02
489   -1.669747e-01
490   -8.370454e-03
491    3.501158e-01
492   -1.799681e-01
493   -1.803171e-01
494    1.805999e-01
495   -1.805006e-01
496   -3.484334e-01
497    1.805307e-01
498   -1.805261e-01
499   -1.261137e-02
500    3.484429e-01
501    3.484428e-01
502   -1.261178e-02
503    1.679155e-01
504    1.805273e-01
505    1.805273e-01
506   -3.484428e-01
507   -1.261180e-02
508   -1.805273e-01
509    3.484428e-01
510    3.484428e-01
511    1.805273e-01
512    1.679155e-01
513    2.444423e-13
514    1.261180e-02
515    1.261180e-02
516    1.679155e-01
517    1.805273e-01
518   -1.805273e-01
519   -1.261180e-02
520    1.805273e-01
521    6.575026e-17
522   -1.261180e-02
523    3.484428e-01
524   -1.805273e-01
525    1.078356e-18
526   -1.261180e-02
527    1.805273e-01
528   -1.679155e-01
529    3.484428e-01
530    1.805273e-01
531    3.484428e-01
532    1.805273e-01
533   -1.805273e-01
534    1.038028e-22
535    1.805273e-01


In [112]:
test['pred'] = sarimax_forecast


C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\1101767578.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
import plotly.express as px
fig = px.line(train, x="Date", y="Change", markers=True)
fig.add_scatter(x=test["Date"], y=test["Change"], mode="lines", name="Test Data",line_color="red")
fig.add_scatter(x=test["Date"], y=test["pred"], mode="lines", name="Predicted Data", line_color="green")

fig.show()

In [114]:
import plotly.graph_objects as go


fig = go.Figure()


fig.add_scatter(x=test["Date"], y=test["Change"], mode="lines", name="Test Data", line_color="red")


fig.add_scatter(x=test["Date"], y=test["pred"], mode="lines", name="Predicted Data", line_color="green")

fig.show()

In [115]:
exog_vars = ['INV_SENT', 'EGX_SENT']


In [116]:
# cross validation manually as this is a forecasting experiment
date_str = train.Date.max()#end date of train data
initial_date = datetime.strptime(date_str, '%Y-%m-%d')
all_mse = []
for i in range(10):
    training_end_date = initial_date - relativedelta(months=(10 - i))
    
    testing_start_date =( training_end_date)
    testing_end_date = (testing_start_date + relativedelta(months=1))
    
    training_end_date = training_end_date.strftime('%Y-%m-%d')
    testing_start_date = testing_start_date.strftime('%Y-%m-%d')
    testing_end_date = testing_end_date.strftime('%Y-%m-%d')

    
 
    train_vald = train[train['Date']<training_end_date]
    test_vald = train[(train['Date']<=testing_end_date) & (train['Date']>=testing_start_date)]
    
    sarimax_model = SARIMAX(train_vald['Change'], exog=train_vald[exog_vars], order=(2,0,0), seasonal_order=(0, 0, 0, 12))

    sarimax_results = sarimax_model.fit()
    sarimax_forecast = sarimax_results.predict(start=test_vald.index[0], end=test_vald.index[-1], exog=test_vald[exog_vars])
    test_vald['pred'] = sarimax_forecast
    mse = mean_squared_error(test_vald['Change'], test_vald['pred'])

    print(f"Iteration {i + 1}:")
    print(f"  Training period: {training_end_date}")
    print(f"  Testing period:  {testing_start_date}")
    print(f"  Testing period:  {testing_end_date}")
    
    print(mse)  
    all_mse.append(mse)

C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Iteration 1:
  Training period: 2023-02-28
  Testing period:  2023-02-28
  Testing period:  2023-03-28
3.5717856319843007


C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Iteration 2:
  Training period: 2023-03-31
  Testing period:  2023-03-31
  Testing period:  2023-04-30
3.362585977528027


C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Iteration 8:
  Training period: 2023-09-30
  Testing period:  2023-09-30
  Testing period:  2023-10-30
3.8963360057364596


C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Iteration 9:
  Training period: 2023-10-31
  Testing period:  2023-10-31
  Testing period:  2023-11-30
2.5530599228049793
Iteration 10:
  Training period: 2023-11-30
  Testing period:  2023-11-30
  Testing period:  2023-12-30
2.5139584414948843


C:\Users\Farah\AppData\Local\Temp\ipykernel_1696\635929165.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [117]:
sum = 0
for i in all_mse:
    sum = sum+i

In [118]:
sum/10

2.1283215476436994

In [119]:
# more measuring matrics
from sklearn.metrics import mean_absolute_error, mean_squared_error


mae = mean_absolute_error(test['Change'], test['pred'])

rmse = mean_squared_error(test['Change'], test['pred'], squared=False)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Absolute Error (MAE): 1.7335820147927519
Root Mean Squared Error (RMSE): 2.436759372268111
